In [35]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F


In [2]:
'''
Unet based model for detecting salt regions in seismic images
Input:
Image 101x101 [0,1]
Transform:
None(for now)
Output:
101x101 matrix with pixel wise confidence values for salt presence

Structure:
DN: A FC layer to create a kernel that applies on the images
UN: A  unet image classifier

'''


'\nUnet based model for detecting salt regions in seismic images\nInput:\nImage 101x101 [0,1]\nTransform:\nNone(for now)\nOutput:\n101x101 matrix with pixel wise confidence values for salt presence\n\nStructure:\nDN: A FC layer to create a kernel that applies on the images\nUN: A  unet image classifier\n\n'

In [64]:
class DN(nn.Module):
    '''
    @IN
    img: A tensor representing the grayscaled image; shape:[batch_size,1,101,101]
    d: The z index of the image
    
    @OUT
    shape:[batch_size,1,101,101] tensor representing the features found by kernel(3*3) created from depth value
    '''
    def __init__(self):
        super(DN,self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(4,9,bias=False),
                                   nn.ReLU())
        self.layer2 = nn.Sequential(nn.Linear(9,9,bias=True),
                                   nn.Sigmoid())
        self.conv1 = nn.Conv2d(1,1,kernel_size=3,stride=1,padding=1,bias=False)
        self.non_linear1 = nn.ReLU()
    def forward(self,x,d):
        ker = torch.from_numpy(np.array([d,d**2,d**3,1])).float()
        ker = self.layer1(ker)
        ker = self.layer2(ker).view(1,1,3,3)
        self.conv1.weight = nn.Parameter(ker)
        x = self.non_linear1(self.conv1(x))
        return x

In [65]:
x = torch.randn(10,1,101,101)
y = torch.randn(10,1)
model =DN()
model(x,45).shape

torch.Size([10, 1, 101, 101])

In [66]:
class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True,up_size=None):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            if up_size:
                self.up = nn.Upsample(size= up_size, mode='bilinear', align_corners=True)
            else:
                self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x1.size()[2] - x2.size()[2]
        diffY = x1.size()[3] - x2.size()[3]
        x2 = F.pad(x2, (diffX // 2 , int(diffX / 2),
                        diffY // 2, int(diffY / 2)))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x
class conv3d(nn.Module):
    '''
    Takes a image with given number of channels and performs 3d convolution and then returns the image as 2d
    '''
    def __init__(self, in_ch, out_ch):
        super(conv3d, self).__init__()
        self.conv = nn.Conv3d(1, 10, 5)

    def forward(self, x):
        x = self.conv(x)
        return x
        

In [67]:
class UNet(nn.Module):
    def __init__(self,debug=False):
        super(UNet, self).__init__()
        self.debug = debug
        self.inc = inconv(1, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256,up_size = (12,12))
        self.up2 = up(512, 128,up_size = (25,25))
        self.up3 = up(256, 64,up_size = (50,50))
        self.up4 = up(128, 64,up_size = (101,101))
        self.outc = outconv(64, 1)

    def forward(self, x):
        x.unsqueeze_(1)
        if self.debug: print(x.shape)
        x1 = self.inc(x)
        if self.debug: print(x1.shape)
        x2 = self.down1(x1)
        if self.debug: print(x2.shape)
        x3 = self.down2(x2)
        if self.debug: print(x3.shape)
        x4 = self.down3(x3)
        if self.debug: print(x4.shape)
        x5 = self.down4(x4)
        if self.debug: print(x5.shape)
        if self.debug: print('before up')
        x = self.up1(x5, x4)
        if self.debug: print(x.shape)
        x = self.up2(x, x3)
        if self.debug: print(x.shape)
        x = self.up3(x, x2)
        if self.debug: print(x.shape)
        x = self.up4(x, x1)
        if self.debug: print(x.shape)
        x = self.outc(x)
        if self.debug: print(x.shape)
        return x.squeeze(1)

In [58]:
model = UNet(debug=True)
x = torch.randn(10,101,101)
print(model(x).shape)

torch.Size([10, 1, 101, 101])
torch.Size([10, 64, 101, 101])
torch.Size([10, 128, 50, 50])
torch.Size([10, 256, 25, 25])
torch.Size([10, 512, 12, 12])
torch.Size([10, 512, 6, 6])
before up


C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


torch.Size([10, 256, 12, 12])
torch.Size([10, 128, 25, 25])
torch.Size([10, 64, 50, 50])
torch.Size([10, 64, 101, 101])
torch.Size([10, 1, 101, 101])
torch.Size([10, 101, 101])


In [34]:
torch.__version__

'0.4.1'

In [74]:
class UDNet(nn.Module):
    def __init__(self):
        super(UDNet,self).__init__()
        self.dn = DN_()
        self.un1 = UNet(debug=True)
        self.un2 = UNet(debug=True)
    def forward(self,x,d):
        x = self.un1(x)
        x.unsqueeze_(1)
        x = self.dn(x,d)
        x.squeeze_(1)
        x = self.un2(x)
        return x

In [ ]:
model = UDNet()
x = torch.randn(10,101,101)
print(model(x,2).shape)

torch.Size([10, 1, 101, 101])
torch.Size([10, 64, 101, 101])
torch.Size([10, 128, 50, 50])
torch.Size([10, 256, 25, 25])
torch.Size([10, 512, 12, 12])
torch.Size([10, 512, 6, 6])
before up


C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


torch.Size([10, 256, 12, 12])
torch.Size([10, 128, 25, 25])
torch.Size([10, 64, 50, 50])
torch.Size([10, 64, 101, 101])
torch.Size([10, 1, 101, 101])
torch.Size([10, 1, 101, 101])
torch.Size([10, 64, 101, 101])
torch.Size([10, 128, 50, 50])
torch.Size([10, 256, 25, 25])
torch.Size([10, 512, 12, 12])
torch.Size([10, 512, 6, 6])
before up
